#  <span style="color:darkblue"> Initial Boundary Value PDEs </span>

*Supplemental material (Julia) for Chapters 7 of "Numerical Methods and Chemical Engineering Applications" by Dorfman and Daoutidis*

<hr style="border:6px solid black"> </hr>

# Learning Objectives

- Be able to recognize Laplace & Poisson equations
- Describe the basic steps involved in the Method of Lines (MOL)
- Apply knowledge of how discretize interior nodes
- Apply knowledge of how to discretize exterior nodes for each boundary condition (Neumann, Robin, Dirichlet)
- Solve a 1D system of PDEs using MOL with explicit Euler's method.
- Formulate the system of ODEs created with MOL by interlacing variables on a rectangular grid.
- Formulate the implicit Euler's method

<hr style="border:6px solid black"> </hr>

# Simulating a graded catalyst bed

Process engineers are often engaged in tasks centered around increasing the economic productivity of process equipment while ensuring safe operation. For the commodity chemical industry, increased conversion of reactant to product remains a desired outcome. One of the most common used continuously operating reactor in this sector is that of the catalytic packed bed reactor. In this configuration, a tubular reactor is employed. This tube is packed with catalyst covered particles and the reactant flows over the catalyst bed. In many cases, a large number of tubes will be used in parallel cooled by the same jacket. We'll just consider the single tube variant here. A simplifed depiction is given below. 

<img src="reactor_diagram.png" width="600">


One important industrial chemical is that ortho-xylene which is used to produce phthalic anhydride a common plastizer. 
The desired reaction is:
<img src="react1edited.png" width="500">

However, two underdesirable side reactions are also known to occur:

<img src="react2edited.png" width="500">

<img src="react3edited.png" width="500">

The reactor is typically operated such at a hotspot occurs in the reactor, the reactor is stable (and in turn safe), and a high degree of selectivity and conversion is achieved. A number of process parameters including pressure, temperature, residence time, cooling rate, and feed composition may be manipulated to improve reactor productivity, selectivity, conversion, or overall profitability. We'll build a simulation (based on the model in this [paper](https://pubs.acs.org/doi/abs/10.1021/ie4005699).) to investigate the how using a graded catalyst may impact reactor performance. That is to say a reactor in which differing zones are loaded with catalyst of differing activity levels. This can allow the operator to raise the effective reactor temperature while preventing thermal runaway and maintaining adequate selectivity.

## A basic descriptive model

A two-dimensional steady-state pseudohomogeneous packed bed reactor equations consist of a mass balance and an energy balance:

\\[ Q\frac{\partial c_j}{\partial V} = \frac{D}{R}\frac{\partial}{\partial R}\left(R\frac{\partial c_j}{\partial R}\right) + \sum_{i}{\nu_{ij}r_i(c,T)}  \\]

\\[\sum_{i}{f_{i}c_{pj}}\frac{\partial T}{\partial V} = \frac{\Lambda}{R}\frac{\partial}{\partial R}\left(R\frac{\partial T}{\partial R}\right) + \sum_{i}{(-\Delta H_{i})r_i(c,T)}  \\]

For a full review of different packed bed reactor models, the reader is directed to [this reference](https://ris.utwente.nl/ws/portalfiles/portal/6073612/t0000040.pdf).

<div class="alert alert-block alert-info">
<b>Form of the above equations:</b> The above equations are coupled convection-diffusion equations. One of the most commonly encountered forms in fluid mechanics.
</div>

<div class="alert alert-block alert-warning">
<b>Limiting cases:</b> XXXX.
</div>

We assume a sufficient cooling flow exists to hold the reactor wall temperature constant at a fixed value of $T^c$. Then we can write a symmetry and cooling () boundary condition:

\\[ Q\frac{\partial c_j}{\partial V} = \frac{D}{R}\frac{\partial}{\partial R}\left(R\frac{\partial c_j}{\partial R}\right) + \sum_{i}{\nu_{ij}r_i(c,T)}  \\]

## Further model simplification

- The radial diffusion in this reactor is typically quite fast (radial concentration gradients can be neglected).

\\[ Q\frac{\partial c_j}{\partial V} = \sum_{i}{\nu_{ij}r_i(c,T)}  \\]

- As the reaction is highly exothermic, a large amount of cooling at the wall is necessary. As such thermal gradients at the wall of the reactor may be high (cannot neglect radial temperature differences).

\\[\left.\frac{dT}{dR}\right\vert_{R=0} = 0 \qquad \qquad \left.\frac{dT}{dR}\right\vert_{R=R_t} = -Bi(T - T^c) \\]

<div class="alert alert-block alert-info">
<b>Note:</b> The wall boundary condition ($R = R_t$) is of the Robin type and the symmetry condition ($R = 0$) is of the Neumann type.
</div>

- Assume the inlet stream is well-mixed.
\\[\begin{align} f_j(0) &= f_{j,in} \\
     T(0) &= T_{in} \end{align}\\]
     
<div class="alert alert-block alert-info">
<b>Note:</b> The inlet conditions are examples of Dirichlet boundary conditions.
</div>

<div class="alert alert-block alert-warning">
<b>Activity!</b> If the type of the boundary condition isn't immediate obvious take a moment and see if you can rearrange these equations into one of the following forms:
- X
- Y
- Z
</div>

- All reactions are treated as pseudo-first-order. Each reaction rate may be written with relation to it's partial pressure

\\[r_1 = (\sigma P_{O_2}\rho_s) k_1 P_{OX} \\]
\\[r_2 = (\sigma P_{O_2}\rho_s) k_2 P_{PA} \\]
\\[r_3 = (\sigma P_{O_2}\rho_s) k_3 P_{OX} \\]

The rate constant can be calculated using the Arrhenius relationship as such

\\[k_i = k_i^r \exp{\frac{E^r_i(T - T^r)}{T R_g T^r}}, \qquad i = \{1, 2, 3\} \\]

Individual component rates of change may then be written as:

\\[\begin{align} r_{OX} &= -r_1 + r_3 \\
r_{PA} &= r_1 - r_2 \\
r_{H_2 O} &= 3r_1 + 2r_2 + 5r_3 \\
r_{O_2} &= -3r_1 - 7.5r_2 + 10.5r_3 \\
r_{CO_2} &= 8r_2 + 8r_3 \\
r_{N_2} &= 0
\end{align}\\]

## Method of Lines Derivation

<hr style="border:6px solid black"> </hr>

We'll now make use 

Unlike typically combustion processes in which a gaseous oxygen source is used the

Chemical looping combustion: https://www.sciencedirect.com/science/article/abs/pii/S1750583613001151

In [ ]:
# Input model parameters

k_ref = [6.519E-2; 5.698E-3; 6.442E-3]   # Reaction rate at refence temperature
E_ref = [113.57; 129.71; 119.68]         # Activation energy at reference temperature
T_ref = 600                              # Reference temperature
Rg =                                     #
catalyst_density = 1300                  # Density of packed catalytic material (kg/m^3)
heat_dispersion = 7.3871                 # heat dispersion coefficient(kJ/m h K)

P0 = 1.7                                 # total atmospheres of pressure
biot_number = 0.8                        # Biot number, Bi (dimensionless)
reactor_length = 9                       # length of reactor (m)
reactor_diam = 0.0254                    # diameter of reactor (m)

feed_rate = 0.174                        # total molar feed rate (mol/h) 
feed_mole_frac = [0.011; 0.208; 0.781]   # mole fractionj


<hr style="border:6px solid black"> </hr>

# Questions for reflection
- For a simple, what factors influence whether a system is numerically stable?
- For IVP-PDEs, the problem size can grow quite rapidly. For a 3D systems, using 20 discretization points in each dimension leads to an 8000-by-8000 system while 256 points leads to a ~16 million by ~16 million system. As illustrated by [Jaroudi, et al.](https://www.tandfonline.com/doi/full/10.1080/00207160.2019.1613526?af=R), a large number of such systems can be solved in a few hours on standard desktop. In light of this fact, how do you think the resulting linear systems formulated and solved?
- What are some other questions that you'd expect to be important when design a chemical looping combustion system? Can you provide some ideas of how numerical methods could be used to help answer these?

<hr style="border:6px solid black"> </hr>